In [2]:
from huggingface_hub import login

# Cargamos el token de HuggingFace que lo tenemos en un fichero oculto e iniciamos sesión

with open("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/huggingfaceToken.txt", "r") as file:
    hf_token = file.read().strip()

login(hf_token)

In [6]:
import torch
torch.cuda.empty_cache()


In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
import torch

# === 1. CARGA DE DATOS ===
df = pd.read_csv("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv")
df = df.rename(columns={"label_task_3_1_merged": "label"})
df["text"] = df["text"].astype(str)
df["label"] = df["label"].astype(int)

# === 2. FORMATO DE INSTRUCCIÓN ===
def format_prompt(row):
    return f"""### Instrucción:
Analiza la siguiente transcripción de video de tiktok a texto para determinar si es misógino o no.
Devuelve exclusivamente la etiqueta "YES" si consideras la transcripción misógina o "NO" si no lo es, sin texto adicional.
Ten en cuenta los posibles detalles que pueden hacer que determines una transcripción como misógina ("YES") o no ("NO")

### Texto:
{row['text']}

### Respuesta:
{"YES" if row['label'] == 1 else "NO"}"""

df["prompt"] = df.apply(format_prompt, axis=1)

# === 3. CONVERSIÓN A HUGGINGFACE DATASET ===
dataset = Dataset.from_pandas(df[["prompt"]])

# === 3.5 LOGGIN EN HUGGINGFACE ===

# Cargamos el token de HuggingFace que lo tenemos en un fichero oculto e iniciamos sesión

with open("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/huggingfaceToken.txt", "r") as file:
    hf_token = file.read().strip()

login(hf_token)

# === 4. CARGA DEL TOKENIZER Y MODELO ===
model_name = "meta-llama/Llama-3.2-3B-Instruct"  # Cambia por cualquier LLM compatible
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # obligatorio para muchos LLMs

# # Definir una clase de procesamiento personalizada
# class CustomProcessing:
#     def __init__(self, tokenizer):
#         self.tokenizer = tokenizer

#     def __call__(self, examples):
#         return self.tokenizer(
#             examples["text"],
#             truncation=True,
#             padding="max_length",
#             max_length=512,
#         )

# # Crear una instancia de la clase de procesamiento
# processing_instance = CustomProcessing(tokenizer)

# === 5. TOKENIZACIÓN ===
def tokenize(example):
    return tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=512)

dataset = dataset.map(tokenize)

# === 6. CARGA DEL MODELO CON LORA (para entrenamiento más ligero) ===
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # depende del modelo base
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

# === 7. ARGUMENTOS DE ENTRENAMIENTO ===
training_args = TrainingArguments(
    output_dir="./llm_classification_output",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="no",
    report_to="none",
)

# # Configuración del entrenamiento
# sft_config = SFTConfig(
#     output_dir="./llm_classification_output",
#     per_device_train_batch_size=2,
#     num_train_epochs=3,
#     fp16=True,
#     logging_steps=10,
#     save_strategy="no",
#     report_to="none",
#     max_seq_length=512,  # Especifica la longitud máxima de secuencia aquí
# )

# === 8. ENTRENAMIENTO CON SFTTrainer ===
trainer = SFTTrainer(
    model=model,
    processing_class=processing_instance,    
    train_dataset=dataset,
    peft_config=peft_config,
    args=tokenizer,
    formatting_func=lambda example: example["prompt"],
)

trainer.train()


NameError: name 'login' is not defined

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
import torch

# === 1. CARGA DE DATOS ===
df = pd.read_csv("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv")
df = df.rename(columns={"label_task_3_1_merged": "label"})
df["text"] = df["text"].astype(str)
df["label"] = df["label"].astype(int)

# === 2. FORMATO DE INSTRUCCIÓN ===
def format_prompt(row):
    return f"""### Instrucción:
Decide si el siguiente mensaje tiene contenido sexista. Responde solo con YES o NO.

### Texto:
{row['text']}

### Respuesta:
{"YES" if row['label'] == 1 else "NO"}"""

df["prompt"] = df.apply(format_prompt, axis=1)

# === 3. CONVERSIÓN A HUGGINGFACE DATASET ===
dataset = Dataset.from_pandas(df[["prompt"]])

# # === 3.5 LOGGIN EN HUGGINGFACE === # ESTO TIENE QUE SER EJECUTADO EN UNA CELDA APARTE PORQUE SI NO FALLA

# # Cargamos el token de HuggingFace que lo tenemos en un fichero oculto e iniciamos sesión

# with open("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/huggingfaceToken.txt", "r") as file:
#     hf_token = file.read().strip()

# login(hf_token)

# === 4. CARGA DEL TOKENIZER Y MODELO ===
model_name = "meta-llama/Llama-3.2-3B-Instruct"  # Cambia por cualquier LLM compatible
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # obligatorio para muchos LLMs

# === 5. TOKENIZACIÓN ===
def tokenize(example):
    return tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=512)

dataset = dataset.map(tokenize)

# === 6. CARGA DEL MODELO CON LORA (para entrenamiento más ligero) ===
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # depende del modelo base
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

# === 7. ARGUMENTOS DE ENTRENAMIENTO ===
training_args = TrainingArguments(
    output_dir="./llm_classification_output",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="no",
    report_to="none",
)

# === 8. ENTRENAMIENTO CON SFTTrainer ===
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args,
    max_seq_length=512,
    formatting_func=lambda example: example["prompt"],
)

trainer.train()


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
/tmp/ipykernel_855332/3225278249.py:78: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'max_seq_length'

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer
import pandas as pd
import torch

# ================================
# 1. Configuración general
# ================================
model_name = "meta-llama/Llama-3.2-1B-Instruct"  # Cambia por cualquier LLM compatible

# Usa la GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"

# ================================
# 2. Carga y preparación de datos
# ================================
df = pd.read_csv("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv")
df = df.rename(columns={"label_task_3_1_merged": "label"})

# Prepara el prompt para clasificación binaria
def format_prompt(row):
    return f"""### Instrucción:
    Decide si el siguiente mensaje tiene contenido sexista. Responde solo con YES o NO.
### Texto:
{row['text']}

### Respuesta:
{"YES" if row['label'] == 1 else "NO"}"""

df["prompt"] = df.apply(format_prompt, axis=1)
df["label"] = df["label"].astype(str)  # Para generar el texto como "0" o "1"
df["completion"] = df["label"]

# Solo se usa la columna prompt como entrada
dataset = Dataset.from_pandas(df[["prompt", "completion"]])

# ================================
# 3. Carga del modelo y tokenizer
# ================================
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # o usa 8bit si falla
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name
# ).to("cuda").half()


# ================================
# 4. Tokenización
# ================================
def formatting(example):
    return f"{example['prompt']} {example['completion']}"

def preprocess(example):
    return tokenizer(
        formatting(example),
        padding="max_length",
        truncation=True,
        max_length=512,
    )

tokenized_dataset = dataset.map(preprocess)

# ================================
# 5. Argumentos de entrenamiento
# ================================
training_args = TrainingArguments(
    output_dir="./outputs_llm",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="no",
    report_to="none",
    #fp16=False,
)

# ================================
# 6. Entrenador SFTTrainer
# ================================
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    tokenizer=tokenizer,
)

# ================================
# 7. Entrenamiento
# ================================
trainer.train()


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/tmp/ipykernel_866865/2036825268.py:94: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [4]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# 1. Configuración
model_name = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Carga y formato del dataset
df = pd.read_csv("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv")
df = df.rename(columns={"label_task_3_1_merged": "label"})

def format_prompt(row):
    return f"""### Instrucción:
Decide si el siguiente mensaje tiene contenido sexista. Responde solo con YES o NO.
### Texto:
{row['text']}

### Respuesta:
{"YES" if row['label'] == 1 else "NO"}"""

df["prompt"] = df.apply(format_prompt, axis=1)
df["completion"] = df["label"].astype(str)
dataset = Dataset.from_pandas(df[["prompt", "completion"]])

# 3. Tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 4. Cuantización y carga del modelo
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# 5. LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# 6. Preprocesamiento
def formatting_func(example):
    return f"{example['prompt']} {example['completion']}"

def preprocess(example):
    return tokenizer(
        formatting_func(example),
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(preprocess)

# 7. Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./outputs_llm_peft",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

# 8. Entrenador
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    peft_config=lora_config,
    #dataset_text_field="prompt",
    tokenizer=tokenizer,
    #max_seq_length=512,
)

trainer.train()


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/tmp/ipykernel_866865/4146028071.py:88: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

  0%|          | 0/7524 [00:00<?, ?it/s]

{'loss': 3.9493, 'grad_norm': 1.8642061948776245, 'learning_rate': 4.9933545986177566e-05, 'mean_token_accuracy': 0.33532283602778923, 'epoch': 0.0}
{'loss': 3.9302, 'grad_norm': 3.3567464351654053, 'learning_rate': 4.9867091972355135e-05, 'mean_token_accuracy': 0.34437720543456407, 'epoch': 0.01}
{'loss': 2.8985, 'grad_norm': 2.973392963409424, 'learning_rate': 4.98006379585327e-05, 'mean_token_accuracy': 0.4768193594056763, 'epoch': 0.01}
{'loss': 3.2828, 'grad_norm': 3.5097124576568604, 'learning_rate': 4.973418394471026e-05, 'mean_token_accuracy': 0.4139964528278739, 'epoch': 0.02}
{'loss': 3.0209, 'grad_norm': 3.5024335384368896, 'learning_rate': 4.966772993088783e-05, 'mean_token_accuracy': 0.4818610244917408, 'epoch': 0.02}
{'loss': 2.8589, 'grad_norm': 7.009305000305176, 'learning_rate': 4.9601275917065395e-05, 'mean_token_accuracy': 0.5050734179720882, 'epoch': 0.02}
{'loss': 3.0464, 'grad_norm': 3.1343560218811035, 'learning_rate': 4.953482190324296e-05, 'mean_token_accuracy'

TrainOutput(global_step=7524, training_loss=2.461456713177054, metrics={'train_runtime': 448.7364, 'train_samples_per_second': 16.767, 'train_steps_per_second': 16.767, 'total_flos': 8858877672296448.0, 'train_loss': 2.461456713177054})

In [1]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

# 1. Configuración
model_name = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Carga y formato del dataset
df = pd.read_csv("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv")
df = df.rename(columns={"label_task_3_1_merged": "label"})

# División estratificada
train_df, test_df = train_test_split(df, test_size=0.1, stratify=df["label"], random_state=42)

def format_prompt(row):
    return f"""### Instrucción:
Decide si el siguiente mensaje tiene contenido sexista. Responde solo con "1" si lo es o "0" si no lo es.
### Texto:
{row['text']}

### Respuesta:
{"YES" if row['label'] == 1 else "NO"}"""

# Aplicar formato a ambos splits
for split_df in (train_df, test_df):
    split_df["prompt"] = split_df.apply(format_prompt, axis=1)
    split_df["completion"] = split_df["label"].astype(str)

# Convertir a Dataset HuggingFace
train_dataset = Dataset.from_pandas(train_df[["prompt", "completion", "label"]])
test_dataset = Dataset.from_pandas(test_df[["prompt", "completion", "label"]])

# 3. Tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 4. Cuantización y carga del modelo
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# 5. LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# 6. Preprocesamiento
def formatting_func(example):
    return f"{example['prompt']} {example['completion']}"

def preprocess(example):
    return tokenizer(
        formatting_func(example),
        truncation=True,
        padding="max_length",
        max_length=64,
    )

train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

# 7. Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./outputs_llm_peft",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

# 8. Entrenador
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    peft_config=lora_config,
    tokenizer=tokenizer,
)

trainer.train()




Map:   0%|          | 0/2257 [00:00<?, ? examples/s]

Map:   0%|          | 0/251 [00:00<?, ? examples/s]

/tmp/ipykernel_874602/2190084578.py:100: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/2257 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2257 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2257 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2257 [00:00<?, ? examples/s]

  0%|          | 0/6771 [00:00<?, ?it/s]

{'loss': 3.9265, 'grad_norm': 2.53337025642395, 'learning_rate': 4.992615566386059e-05, 'mean_token_accuracy': 0.3186500116428796, 'epoch': 0.0}
{'loss': 3.4969, 'grad_norm': 2.9931998252868652, 'learning_rate': 4.985231132772117e-05, 'mean_token_accuracy': 0.39275411641839775, 'epoch': 0.01}
{'loss': 3.5968, 'grad_norm': 3.1474406719207764, 'learning_rate': 4.977846699158175e-05, 'mean_token_accuracy': 0.3781121485097704, 'epoch': 0.01}
{'loss': 3.0993, 'grad_norm': 4.0319013595581055, 'learning_rate': 4.9704622655442326e-05, 'mean_token_accuracy': 0.4277257379504517, 'epoch': 0.02}
{'loss': 2.6994, 'grad_norm': 3.061086416244507, 'learning_rate': 4.963077831930291e-05, 'mean_token_accuracy': 0.5201531473749046, 'epoch': 0.02}
{'loss': 2.6263, 'grad_norm': 4.010173797607422, 'learning_rate': 4.955693398316349e-05, 'mean_token_accuracy': 0.5546146603514954, 'epoch': 0.03}
{'loss': 2.8511, 'grad_norm': 2.592853546142578, 'learning_rate': 4.948308964702408e-05, 'mean_token_accuracy': 0.5

TrainOutput(global_step=6771, training_loss=2.295069775189101, metrics={'train_runtime': 410.0615, 'train_samples_per_second': 16.512, 'train_steps_per_second': 16.512, 'total_flos': 8400770288640000.0, 'train_loss': 2.295069775189101})

In [5]:
# 1. Generar texto para cada entrada
from tqdm import tqdm

trainer.model.eval()
trainer.model.to("cuda")
tokenizer.padding_side = "left"

predictions = []
for example in tqdm(test_dataset):
    inputs = tokenizer(example["prompt"], return_tensors="pt", truncation=True, padding=True).to("cuda")
    outputs = trainer.model.generate(**inputs, max_new_tokens=10)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extrae la última línea (respuesta del modelo)
    response = decoded.split("### Respuesta:")[-1].strip()
    predictions.append(1 if "YES" in response.upper() else 0)

# 2. Comparar con los verdaderos
from sklearn.metrics import classification_report

true_labels = test_df["label"].tolist()
print(classification_report(true_labels, predictions, digits=4))


100%|██████████| 251/251 [00:31<00:00,  8.08it/s]

              precision    recall  f1-score   support

           0     1.0000    0.8626    0.9262       131
           1     0.8696    1.0000    0.9302       120

    accuracy                         0.9283       251
   macro avg     0.9348    0.9313    0.9282       251
weighted avg     0.9376    0.9283    0.9281       251



Prediccion sobre el fichero de test

In [8]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# 1. Configuración
model_name = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Carga y formato del dataset de entrenamiento
df = pd.read_csv("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv")
df = df.rename(columns={"label_task_3_1_merged": "label"})

def format_prompt(row):
    return f"""### Instrucción:
Decide si el siguiente mensaje tiene contenido sexista. Responde solo con "1" si lo es o "0" si no lo es.
### Texto:
{row['text']}

### Respuesta:
{"YES" if row['label'] == 1 else "NO"}"""

df["prompt"] = df.apply(format_prompt, axis=1)
df["completion"] = df["label"].astype(str)
train_dataset = Dataset.from_pandas(df[["prompt", "completion"]])

# 3. Tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 4. Cuantización y carga del modelo
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# 5. LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# 6. Tokenización del entrenamiento
def formatting_func(example):
    return f"{example['prompt']} {example['completion']}"

def preprocess(example):
    return tokenizer(
        formatting_func(example),
        truncation=True,
        padding="max_length",
        max_length=64,
    )

train_dataset = train_dataset.map(preprocess)

# 7. Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./outputs_llm_peft",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

# 8. Entrenador
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    peft_config=lora_config,
    tokenizer=tokenizer,
)

trainer.train()

# ================================================
# 9. Cargar y predecir sobre un dataset sin etiquetas
# ================================================

df_test = pd.read_json("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/test/EXIST2025_test_clean.json", orient="index")
df_test["prompt"] = df_test["text"].apply(lambda x: f"""### Instrucción:
Decide si el siguiente mensaje tiene contenido sexista. Responde solo con "1" si lo es o "0" si no lo es.
### Texto:
{x}

### Respuesta:""")
test_dataset = Dataset.from_pandas(df_test[["prompt"]])

# Tokenización
def preprocess_test(example):
    return tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=64,
    )

test_dataset = test_dataset.map(preprocess_test)

# Predicción
trainer.model.eval()
trainer.model.to(device)
outputs = trainer.model.generate(
    input_ids=torch.tensor(test_dataset["input_ids"]).to(device),
    attention_mask=torch.tensor(test_dataset["attention_mask"]).to(device),
    max_new_tokens=10
)

decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
df_test["prediction"] = decoded_outputs

# Guardar resultados
df_test[["id_EXIST", "text", "prediction"]].to_csv("predicciones_test.csv", index=False)


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/tmp/ipykernel_874602/3064019536.py:88: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

  0%|          | 0/7524 [00:00<?, ?it/s]

{'loss': 3.7134, 'grad_norm': 1.8114827871322632, 'learning_rate': 4.9933545986177566e-05, 'mean_token_accuracy': 0.3539391666195547, 'epoch': 0.0}
{'loss': 3.6674, 'grad_norm': 3.1243834495544434, 'learning_rate': 4.9867091972355135e-05, 'mean_token_accuracy': 0.3654013652699131, 'epoch': 0.01}
{'loss': 2.7064, 'grad_norm': 2.8207504749298096, 'learning_rate': 4.98006379585327e-05, 'mean_token_accuracy': 0.4988299668119701, 'epoch': 0.01}
{'loss': 3.1156, 'grad_norm': 3.1351206302642822, 'learning_rate': 4.973418394471026e-05, 'mean_token_accuracy': 0.4372510237269432, 'epoch': 0.02}
{'loss': 2.8168, 'grad_norm': 3.3498618602752686, 'learning_rate': 4.966772993088783e-05, 'mean_token_accuracy': 0.5133176623593839, 'epoch': 0.02}
{'loss': 2.5475, 'grad_norm': 3.8901798725128174, 'learning_rate': 4.9601275917065395e-05, 'mean_token_accuracy': 0.5662161956623256, 'epoch': 0.02}
{'loss': 2.7913, 'grad_norm': 3.310295343399048, 'learning_rate': 4.953482190324296e-05, 'mean_token_accuracy':

Map:   0%|          | 0/674 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [11]:
from tqdm import tqdm

# Generación en lotes pequeños si es necesario
generated_outputs = []
batch_size = 2  # ajusta si tienes poca memoria

for i in tqdm(range(0, len(test_dataset), batch_size)):
    batch = test_dataset.select(range(i, min(i + batch_size, len(test_dataset))))
    inputs = tokenizer(
        batch["prompt"],
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=64
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            temperature=0.0
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    generated_outputs.extend(decoded)

# Limpieza y extracción de respuestas
def extract_label(text):
    text = text.strip().upper()
    if "YES" in text:
        return 1
    elif "NO" in text:
        return 0
    else:
        return -1  # valor desconocido

predicted_labels = [extract_label(output) for output in generated_outputs]

# Añadir al DataFrame
df_test["prediction_raw"] = generated_outputs
df_test["predicted_label"] = predicted_labels

# Guardar
df_test[["id_EXIST", "text", "predicted_label"]].to_csv("/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/LLM_cuaderno/predicciones_test.csv", index=False)


  0%|          | 0/337 [00:00<?, ?it/s]/home/adrian/anaconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/adrian/anaconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
100%|██████████| 337/337 [00:49<00:00,  6.86it/s]


In [14]:
df_test = pd.read_csv("/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/LLM_cuaderno/predicciones_test.csv")

df_test.value_counts('predicted_label')

predicted_label
0    623
1     51
Name: count, dtype: int64

In [ ]:
df_test_json = df_test.to_json()

df_test_json

In [15]:
import pandas as pd
import json

# Cargar el CSV
#df = pd.read_csv("predicciones_test.csv")

# Convertir los valores
output = []
for _, row in df_test.iterrows():
    entry = {
        "test_case": "EXIST2025",
        "id": str(row["id_EXIST"]),
        "value": "YES" if row["predicted_label"] == 1 else "NO"
    }
    output.append(entry)

# Guardar como JSON
with open("/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/LLM_cuaderno/predicciones_formato_EXIST2025.json", "w") as f:
    json.dump(output, f, indent=2)

print("✅ Archivo guardado como 'predicciones_formato_EXIST2025.json'")


✅ Archivo guardado como 'predicciones_formato_EXIST2025.json'


In [17]:
df_json = pd.read_json('/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/LLM_cuaderno/predicciones_formato_EXIST2025.json')

df_json.value_counts('value')

value
NO     623
YES     51
Name: count, dtype: int64

Formato de la subida *PyEvall*

In [18]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

predictions = "/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/exist2025_I2C-UHU-Sirius/task3_1_hard_I2C-UHU-Sirius_1/Llama-3.2-1B-Instruct.json"
#gold = "/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/exist2025_I2C-UHU-Sirius/task3_1_hard_I2C-UHU-Sirius_1/prediction.json"
gold = "/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/evaluation/golds/EXIST2025_training_task3_1_gold_hard.json"
test = PyEvALLEvaluation()
params= dict()
params[PyEvALLUtils.PARAM_REPORT]= PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED
metrics=["ICM", "ICMNorm" ,"FMeasure"]
report= test.evaluate(predictions, gold, metrics, **params)
report.print_report()

2025-05-18 14:18:46,153 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM', 'ICMNorm', 'FMeasure']
2025-05-18 14:18:46,230 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 14:18:46,487 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Normalized evaluation method
2025-05-18 14:18:46,488 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 14:18:46,741 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-18 14:18:47,183 - pyevall.metrics.metrics - INFO -             evaluate() - Executing fmeasure evaluation method
cargado 29
{
  "metrics": {
    "ICM": {
      "name": "Information Contrast model",
      "acronym": "ICM",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "average